In [18]:
# imports
import numpy as np
import pandas as pd
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

ModuleNotFoundError: No module named 'xgboost'

In [19]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.fc1 = nn.Linear(42, 84)
#         self.relu = nn.ReLU()
#         self.fc2 = nn.Linear(84, 1)

#     def forward(self, x):
#         x = self.fc1(x)
#         x = self.relu(x)
#         x = self.fc2(x)
#         return x

In [20]:
# # boosted decision tree
# class BoostedDecisionTree:
#     def __init__(self, num_of_trees, max_depth=1):
#         self.trees = []
#         self.weights = []
#         self.num_of_trees = num_of_trees
#         self.max_depth = max_depth

#     # try and see how random forest compares
#     # try using class weights here
#     # try gini and see training time change
#     def fit(self, X, y, init_weights):
#         N = X.shape[0]
#         w = init_weights

#         for m in range(self.num_of_trees):
#             tree = DecisionTreeClassifier(max_depth=self.max_depth, criterion="gini")

#             norm_w = np.exp(w)
#             norm_w /= norm_w.sum()
            
#             tree.fit(X, y, sample_weight=norm_w)
#             yhat = tree.predict(X)

#             eps = norm_w.dot(yhat != y) + 1e-20
#             alpha = (np.log(1 - eps) - np.log(eps)) / 2

#             w = w - alpha * y * yhat

#             self.trees.append(tree)
#             self.weights.append(alpha)

#     def predict(self,X):
#         # try fixing this as current predictions arounf 48-52% are overwritten
#         y = np.zeros(X.shape[0])
#         for (tree, alpha) in zip(self.trees, self.weights):
#             y = y + alpha * tree.predict(X)
#         y = y - y.min()
#         y = [round(num) for num in y]
#         return y

In [2]:
# getting the data
df = pd.read_csv('data/train_preprocessed_data.csv')
pd.set_option('max_columns', None)
df['loan_paid'].value_counts()

1    776024
0    191895
Name: loan_paid, dtype: int64

In [3]:
# setting up data
y_train = df['loan_paid'].to_numpy()
x_train = df.drop(columns=['loan_paid']).to_numpy()

In [4]:
# fixing numpy conversion issues
x_train[np.isinf(x_train)] = 0
print(np.where(np.isinf(x_train)))

print(np.where(np.isinf(y_train)))

(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64),)


In [24]:
# logistic regression model
# lg = LogisticRegression(random_state=0).fit(x_train, y_train)

In [13]:
# random forest model
rfc1 = GradientBoostingClassifier(n_estimators=50, max_depth=2, subsample=1)
rfc1.fit(x_train, y_train)

GradientBoostingClassifier(max_depth=2, n_estimators=50, subsample=1)

In [14]:
rfc2 = GradientBoostingClassifier(n_estimators=150, max_depth=2, subsample=1)
rfc2.fit(x_train, y_train)

GradientBoostingClassifier(max_depth=2, n_estimators=150, subsample=1)

In [15]:
rfc3 = GradientBoostingClassifier(n_estimators=300, max_depth=2, subsample=1)
rfc3.fit(x_train, y_train)

KeyboardInterrupt: 

In [26]:
rfc4 = GradientBoostingClassifier(n_estimators=500, max_depth=2, subsample=1)
rfc4.fit(x_train, y_train)

GradientBoostingClassifier(max_depth=2, n_estimators=500, subsample=1)

In [ ]:
rfc5 = GradientBoostingClassifier(n_estimators=1000, max_depth=2, subsample=1)
rfc5.fit(x_train, y_train)

In [31]:
# # getting stored cluster_count and k_means model
# with open('models/k_means.pickle', 'rb') as handle:
#     k_means = pickle.load(handle)
    
# with open('data/cluster_count.pickle', 'rb') as handle:
#     cluster_count = pickle.load(handle)

In [32]:
# # generating init_weights
# init_weights = []
# total_count = sum(cluster_count)
# cluster_pred = k_means.predict(x_train)

# for pred in cluster_pred:
#     init_weights.append(cluster_count[pred] / total_count)

# init_weights = np.array(init_weights)
# init_weights = init_weights / sum(init_weights)
# init_weights

In [33]:
# # training the data
# # maybe overfitting? reduce # of trees necessary
# model = BoostedDecisionTree(250, 1)
# model.fit(x_train, y_train, init_weights)

In [21]:
# getting validation input
x_val = pd.read_csv('data/predict_preprocessed_data.csv')
x_val_id = x_val['ID']
x_val = x_val.drop(columns=['ID'])

In [22]:
# converting to np array
x_val = x_val.to_numpy()
print(np.where(np.isinf(x_val)))
x_val[np.isinf(x_val)] = 0
np.where(np.isinf(x_val))

(array([140519]), array([12]))


(array([], dtype=int64), array([], dtype=int64))

In [36]:
# # predicting using logistic regression
# pred_probs = lg.predict(x_val)
# pred_probs

In [21]:
DT = GradientBoostingRegressor(n_estimators=500, max_depth=3, learning_rate=0.1)
DT.fit(x_train, y_train)

GradientBoostingRegressor(n_estimators=500)

In [23]:
pred = DT.predict(x_train)

In [26]:
preds = []
for p in pred:
    if (p >= .5):
        preds.append(1)
    else:
        preds.append(0)

In [28]:
correct = 0
for p, c in zip(preds, y_train):
    if p == c:
        correct = correct + 1
print(correct / y_train.shape[0])

0.802887


In [37]:
# # predicting using random forest
# pred_probs = rfc.predict(x_val)
# zero = min(pred_probs)
# pred = []
# for prob in pred_probs:
#     if (prob == zero):
#         pred.append(0)
#     else:
#         pred.append(1)
# pred

In [38]:
# net = Net()
# net.fit(x_train, y_train)
# optimizer = torch.optim.Adam(net.parameters(), lr=.001)

# for epoch in 500:
#     optimizer.zero_grad()

#     y_pred = net(x_train)
#     train_loss = my_NLLloss(y_pred, y_train)

#     y_pred = net(x_val)
#     val_loss = my_NLLloss(y_pred, y_val)

#     train_loss.backward()        
#     optimizer.step()
#     # <<< END YOUR CODE

#     # DO NOT MODIFY THE BELOW
#     train_losses.append(train_loss.item())
#     val_losses.append(val_loss.item())

In [23]:
pred = rfc2.predict(x_val)

In [25]:
# creating submission
submission = pd.DataFrame()
submission['ID'] = x_val_id
submission['loan_paid'] = pred
submission.to_csv('submissions/pred.csv', index=None)

In [27]:
r = [rfc4]
r

[GradientBoostingClassifier(max_depth=2, n_estimators=500, subsample=1)]

In [28]:
for rfc in r:
    print(rfc.score(x_train, y_train))

0.8043710269144422
